In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
groq_api_key = os.getenv('GROQ_API_KEY')

In [17]:
from langchain_groq import ChatGroq

model = ChatGroq(model = 'Gemma2-9b-it', groq_api_key = groq_api_key)


In [18]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="")Hi, my name is Marshal and I am a chief AI engineer.])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (258825698.py, line 2)

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content='Hi, my name is Marshal and I am a chief AI engineer.'),
        AIMessage(content="Hello Marshal! It's a pleasure to meet you. \n\nThat's a fascinating title. As a large language model, I'm always interested in learning more about the work that people like you are doing in the field of AI.\n\nWhat kind of projects are you currently working on?  Do you have any interesting challenges you're facing?\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Marshal and that you are a chief AI engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 119, 'total_tokens': 167, 'completion_time': 0.087272727, 'prompt_time': 0.005325637, 'queue_time': 0.160455054, 'total_time': 0.092598364}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0b1100e1-c1dd-45ed-b801-71289c9f5bbf-0', usage_metadata={'input_tokens': 119, 'output_tokens': 48, 'total_tokens': 167})

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [ ]:
config = {"configurable": {"session_id": "chat1"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi, my name is Marshal and I am a chief AI engineer.')],
    config = config
)

In [ ]:
response.content

"Hello Marshal, it's a pleasure to meet you! That's a very impressive title. What exciting things are you working on as Chief AI Engineer? \n\nI'm always curious to learn about the cutting-edge applications of AI.  \n\n"

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config = config
)

AIMessage(content='Your name is Marshal.  You told me at the beginning of our conversation. 😊 \n\n\n\nIs there anything else I can help you with, Marshal?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 171, 'total_tokens': 204, 'completion_time': 0.06, 'prompt_time': 0.010607958, 'queue_time': 0.161771328, 'total_time': 0.070607958}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1be03cf5-63af-448d-a665-d6a1bb313924-0', usage_metadata={'input_tokens': 171, 'output_tokens': 33, 'total_tokens': 204})

In [ ]:
# Change the config ---> session id
config1 = {"configurable": {"session_id": "chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config = config1
)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know!\n"

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey my name is Linux.")],
    config = config1
)

response.content

'Nice to meet you, Linux! What can I do for you today?\n'

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config = config1
)

response.content

'Your name is Linux! 🐧  \n\nI remember that you told me. 😊 \n\nIs there anything else I can help you with?\n'

### Prompt templates

Prompt templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Marshal")]})

AIMessage(content="Hi Marshal, it's nice to meet you! \n\nWhat can I do for you today?  I'm ready for any questions you have. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 30, 'total_tokens': 68, 'completion_time': 0.069090909, 'prompt_time': 0.002127144, 'queue_time': 0.159314384, 'total_time': 0.071218053}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--5c5b365a-738a-4923-a564-db11b04d0cad-0', usage_metadata={'input_tokens': 30, 'output_tokens': 38, 'total_tokens': 68})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [23]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Marshal")],
    config=config
)

response

AIMessage(content="Hi Marshal!  \n\nIt's great to meet you too.  What can I do for you? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 77, 'total_tokens': 104, 'completion_time': 0.049090909, 'prompt_time': 0.004055051, 'queue_time': 0.160637328, 'total_time': 0.05314596}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--60581bf5-5eaf-4bd7-9db0-31c384c81120-0', usage_metadata={'input_tokens': 77, 'output_tokens': 27, 'total_tokens': 104})

In [24]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Marshal. 😊  \n\nIs there anything else I can help you with? \n'

In [25]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [27]:
response=chain.invoke({
    "messages":[HumanMessage(content="Hi My name is Marshal")],
    "language":"Spanish"
    })
response.content

'¡Hola Marshal! ¿En qué puedo ayudarte? 😊 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [29]:
config = {"configurable": {"session_id": "chat4"}}
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Marshal")],"language":"Spanish"},
    config=config
)
response.content

'¡Hola Marshal! 👋  \n\n¿Cómo puedo ayudarte hoy? 😄\n'

In [ ]:
response = with_message_history.invoke(
    {
    "messages": [HumanMessage(content="whats my name?")],
     "language": "Spanish"
    },
    config = config,
)

In [32]:
response.content

'¡Tu nombre es Marshal! 😊  Lo recuerdo.  ¿Qué más te gustaría saber o hacer?  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [33]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/home/marshal/Estudos/IA/Krish_Naik/Generative_AI_course/S22_Introduction_to_Langchain_for_generative_AI/langchain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model

)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences!\n\nWhat's your favorite flavor? 😊🍦\n"

In [35]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2" 😊  '

In [36]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [37]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or personal information about you. So I don't know your name. 😊 \n\nWould you like to tell me?\n"

In [38]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is.  \n\n"